# FLUX.2 Tutorial Notebook

This notebook covers:
1. Basic generation
2. Batch processing
3. Model comparison
4. API integration
5. Custom pipeline wrappers

> Set `DRY_RUN = False` only after local models are configured.

In [ ]:
from pathlib import Path
from flux2.streamlit_adapter import get_adapter

DRY_RUN = True
OUTPUT_DIR = Path('outputs/notebook')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print('DRY_RUN =', DRY_RUN)
print('Output dir:', OUTPUT_DIR.resolve())

In [ ]:
adapter = get_adapter()
model_id = 'flux.2-klein-4b'

if DRY_RUN:
    print(f'[DRY_RUN] Would load model: {model_id}')
else:
    adapter.load(model_id, dtype_str='bf16', cpu_offloading=False, attn_slicing=False)
    print('Model loaded:', model_id)

In [ ]:
prompt = 'A serene mountain landscape at sunset'
request = dict(prompt=prompt, num_steps=4, guidance=3.5, width=768, height=768, seed=42)

if DRY_RUN:
    print('[DRY_RUN] generate request:', request)
else:
    image = adapter.generate(**request)
    out = OUTPUT_DIR / 'basic_generation.png'
    image.save(out)
    print('Saved:', out)
    image

In [ ]:
batch_prompts = [
    'A red sports car in studio lighting',
    'A blue vintage airplane over mountains',
    'A green forest trail with mist'
]

for idx, text in enumerate(batch_prompts, start=1):
    req = dict(prompt=text, num_steps=4, guidance=3.5, width=768, height=768, seed=100 + idx)
    if DRY_RUN:
        print(f'[DRY_RUN] batch {idx}:', req)
    else:
        img = adapter.generate(**req)
        out = OUTPUT_DIR / f'batch_{idx}.png'
        img.save(out)
        print('Saved:', out)

In [ ]:
comparison_prompt = 'A cozy living room interior photo'
models = ['flux.2-klein-4b', 'flux.2-klein-base-4b']

for model in models:
    if DRY_RUN:
        print(f'[DRY_RUN] Would compare with model={model}')
        continue

    adapter.load(model, dtype_str='bf16')
    img = adapter.generate(
        prompt=comparison_prompt,
        num_steps=4,
        guidance=3.5,
        width=768,
        height=768,
        seed=42,
    )
    out = OUTPUT_DIR / f'compare_{model}.png'
    img.save(out)
    print('Saved:', out)

In [ ]:
short_prompt = 'A cat'

if DRY_RUN:
    print('[DRY_RUN] upsample request:', short_prompt)
else:
    upsampled = adapter.upsample_prompt(
        short_prompt,
        backend='openrouter',
        openrouter_model='mistralai/pixtral-large-2411',
    )
    print('Upsampled prompt:
', upsampled)

In [ ]:
def generate_art(prompt: str, model_id: str = 'flux.2-klein-4b', *, seed: int = 42):
    adapter = get_adapter()
    if DRY_RUN:
        return {'mode': 'dry_run', 'model': model_id, 'prompt': prompt, 'seed': seed}

    adapter.load(model_id, dtype_str='bf16')
    image = adapter.generate(
        prompt=prompt,
        num_steps=4,
        guidance=3.5,
        width=768,
        height=768,
        seed=seed,
    )
    out = OUTPUT_DIR / f'api_{seed}.png'
    image.save(out)
    return {'mode': 'real', 'output': str(out)}

generate_art('A cinematic sci-fi city at dawn')

In [ ]:
class SafeGenerationPipeline:
    def __init__(self, adapter):
        self.adapter = adapter

    def run(self, prompt: str, seed: int = 42):
        if DRY_RUN:
            return {'status': 'dry_run', 'prompt': prompt, 'seed': seed}
        try:
            return self.adapter.generate(
                prompt=prompt,
                num_steps=4,
                guidance=3.5,
                width=768,
                height=768,
                seed=seed,
            )
        except Exception as exc:
            return {'status': 'failed', 'error': str(exc)}

pipeline = SafeGenerationPipeline(adapter)
pipeline.run('A minimalist product photo of a wristwatch')

## Next Steps

- Switch `DRY_RUN = False`
- Ensure local model weights are configured
- Re-run cells to produce actual outputs in `outputs/notebook/`